### Load Modules


In [ ]:
# Mount Google drive 
from google.colab import drive
drive.mount('/content/drive')

# Set source of the project folder in drive
source = "<<Location to code dependencies in mounted google drive>>"

# Create a project folder.
!mkdir ./Project
# Copy all project data to this folder.
!cp -R {source} ./Project

# Change directory to projects/code
%cd Project/Code/

# Install required dependencies 
! pip install -q -r ./requirements-yolov3.txt > temp

# Main

In [ ]:
# This code was designed to be ran on Google Colab.

%cd /content/Project/Code/
#@title String fields
video_1 = 'Video1' #@param ["Video1", "Video2", "Video3", "Video4", "Video5", "Video6", "Video7", "Video8" ]
video_2 = 'Video2' #@param ["Video1", "Video2", "Video3", "Video4", "Video5", "Video6", "Video7", "Video8" ]
video_3 = 'Video3' #@param ["Video1", "Video2", "Video3", "Video4", "Video5", "Video6", "Video7", "Video8" ]
# video_4 = 'Video1' #@param ["Video1", "Video2", "Video3", "Video4", "Video5", "Video6", "Video7", "Video8" ]


#@markdown  ---
#@markdown  **Debug**
# Counts = False #@param {type:"boolean"}
Density = True #@param {type:"boolean"}
Plot = False #@param {type:"boolean"}
# Density = False
# Plot = False

#@markdown  ---
# END of FORM

# import necessary packages
import numpy as np
import cv2 as cv
from Modules import yolov3
from Modules import timer
import matplotlib.pyplot as plt


# Prepare input videos paths
input_videos = [
    f"./Data/Test videos/{video}.mp4" 
        for video in [
            video_1, 
            video_2, 
            video_3
        ]
]

# Load all 4 videos of each lane.
frames = []
for video_path in input_videos:
    cap = cv.VideoCapture(video_path)

    ret, frame = cap.read()
    if ret:
        frames.append(
            cv.resize(frame, (416, 416) )
        )
    else:
        print("[ERROR] : Some video(s) not loading properly.")
        raise ValueError

# Plto the lanes using matplotlib if plot is checked
if Plot:
    fig, axs = plt.subplots(1, 3, figsize=(20, 10), sharey=True)    
    axs[0].imshow(frames[0])
    axs[1].imshow(frames[1])
    axs[2].imshow(frames[2])

# Once frames are processed,
# use the detector to get the objects in the frames.
result = yolov3.detect_multiple(frames)

# Extract only the vehicles counts from detection results
counts = []
for i, res in enumerate(result):
    # print(f"Frame {i+1}: ")
    count = {}
    for key in res:
        # print("   ", key, res[key][0])
        count[key] = res[key][0]
    counts.append(count)

# Convert to density
# For each frame get grand total of vehicles
total_counts_per_lane = []
conversion_dict = {
    'truck': lambda x: x*5,
    'bus': lambda x: x*4,
    'car': lambda x: x*2,
    'bike': lambda x: x
}

for count_dict in counts:
    total_count = 0
    for vType, count in count_dict.items():
        total_count += conversion_dict[ vType ] ( count )
    total_counts_per_lane.append(total_count)

if Density:
    print("[DEBUG]: Lane Densities: ")
    for i in range(1, 4):
        print(f"   Lane {i}: ", total_counts_per_lane[i-1])

timer.Timer().analyze(total_counts_per_lane[:3])

